# House 3D

Comandos disponíveis: ???

In [106]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [107]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 768
largura = 1366
qtd_texturas = 30
prox_textura = 1
window = glfw.create_window(largura, altura, "House", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [108]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [109]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [110]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [111]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [112]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [113]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [114]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [115]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [116]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    filepath = 'Objects/' + filename
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filepath, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

In [117]:
glEnable(GL_TEXTURE_2D)
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    texturepath = 'Objects/' + img_textura
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(texturepath)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [118]:
vertices_list = []    
textures_coord_list = []

Esta função adiciona a textura na lista recebida como parâmetro e aumenta o valor do id da textura

In [119]:
def increase_text(text_id):
    global prox_textura
    text_id.append(prox_textura)
    prox_textura += 1

### Vamos carregar cada modelo e definir funções para desenhá-los

In [120]:
modelo = load_model_from_file('terreno/pedra.obj')

print('Processando modelo piso.obj. Vertice inicial:',len(vertices_list))
pedra_text_id = []
pedra_vertices_text = []
ini_pedra = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo piso.obj. Vertice final:',len(vertices_list))
fim_pedra = len(vertices_list)

pedra_vertices_text.append(ini_pedra)
pedra_vertices_text.append(fim_pedra)

load_texture_from_file(prox_textura,'terreno/piso.jpg')
increase_text(pedra_text_id)

Processando modelo piso.obj. Vertice inicial: 0
Processando modelo piso.obj. Vertice final: 6


In [121]:
modelo = load_model_from_file('terreno/grama.obj')

print('Processando modelo grama.obj. Vertice inicial:',len(vertices_list))
grama_text_id = []
grama_vertices_text = []
ini_grama = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo grama.obj. Vertice final:',len(vertices_list))
fim_grama = len(vertices_list)

grama_vertices_text.append(ini_grama)
grama_vertices_text.append(fim_grama)

load_texture_from_file(prox_textura,'terreno/grama.jpg')
increase_text(grama_text_id)

Processando modelo grama.obj. Vertice inicial: 6
Processando modelo grama.obj. Vertice final: 12


In [122]:
modelo = load_model_from_file('casa/casa.obj')

print('Processando modelo casa.obj. Vertice inicial:',len(vertices_list))
casa_vertices_text = []
faces_visited = []
casa_text_id = []
ini_casa = len(vertices_list)
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], 'vertice inicial: ', len(vertices_list))
        faces_visited.append(face[2])
        casa_vertices_text.append(len(vertices_list))
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))
fim_casa = len(vertices_list)
    
casa_vertices_text.append(fim_casa)

load_texture_from_file(prox_textura,'casa/metal_dark_brown.jpg')
increase_text(casa_text_id)
load_texture_from_file(prox_textura,'casa/metal_grey.jpg')
increase_text(casa_text_id)
load_texture_from_file(prox_textura,'casa/wood_brown.jpg')
increase_text(casa_text_id)
load_texture_from_file(prox_textura,'casa/plaster_light_brown.jpg')
increase_text(casa_text_id)
load_texture_from_file(prox_textura,'casa/plaster_sand.jpg')
increase_text(casa_text_id)
load_texture_from_file(prox_textura,'casa/plate_grey.jpg')
increase_text(casa_text_id)
load_texture_from_file(prox_textura,'casa/wood_balls_brown.jpg')
increase_text(casa_text_id)
load_texture_from_file(prox_textura,'casa/foundation_brown_brick.jpg')
increase_text(casa_text_id)
load_texture_from_file(prox_textura,'casa/glass_window.jpg')
increase_text(casa_text_id)

Processando modelo casa.obj. Vertice inicial: 12
metal_dark_brown vertice inicial:  12
metal_grey vertice inicial:  469578
wood_brown vertice inicial:  571176
plaster_light_brown vertice inicial:  580644
plaster_sand vertice inicial:  582819
plate_grey vertice inicial:  583779
wood_balls_brown vertice inicial:  583809
foundation_brown_brick vertice inicial:  584145
glass_window vertice inicial:  585381
Processando modelo casa.obj. Vertice final: 585477


In [123]:
modelo = load_model_from_file('sofa/sofa.obj')

print('Processando modelo sofa.obj. Vertice inicial:',len(vertices_list))
sofa_text_id = []
sofa_vertices_text = []
ini_sofa = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo sofa.obj. Vertice final:',len(vertices_list))
fim_sofa = len(vertices_list)

sofa_vertices_text.append(ini_sofa)
sofa_vertices_text.append(fim_sofa)

load_texture_from_file(prox_textura,'sofa/sofa.jpg')
increase_text(sofa_text_id)

Processando modelo sofa.obj. Vertice inicial: 585477
Processando modelo sofa.obj. Vertice final: 590289


In [124]:
modelo = load_model_from_file('tv/tv.obj')

print('Processando modelo tv.obj. Vertice inicial:',len(vertices_list))
tv_text_id = []
tv_vertices_text = []
ini_tv = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo tv.obj. Vertice final:',len(vertices_list))
fim_tv = len(vertices_list)

tv_vertices_text.append(ini_tv)
tv_vertices_text.append(fim_tv)

load_texture_from_file(prox_textura,'tv/tv.png')
increase_text(tv_text_id)

Processando modelo tv.obj. Vertice inicial: 590289
Processando modelo tv.obj. Vertice final: 1263057


In [125]:
modelo = load_model_from_file('arvore/arvore-1.obj')

print('Processando modelo arvore-1.obj. Vertice inicial:',len(vertices_list))
ini_arvore1 = len(vertices_list)
arvore1_vertices_text = []
arvore1_text_id = []
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], 'vertice inicial: ', len(vertices_list))
        faces_visited.append(face[2])
        arvore1_vertices_text.append(len(vertices_list))
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo arvore.obj. Vertice final:',len(vertices_list))
fim_arvore1 = len(vertices_list)
    
arvore1_vertices_text.append(fim_arvore1)

load_texture_from_file(prox_textura,'arvore/tronco-1.jpg')
increase_text(arvore1_text_id)
load_texture_from_file(prox_textura,'arvore/folha-1.jpg')
increase_text(arvore1_text_id)

Processando modelo arvore-1.obj. Vertice inicial: 1263057
Materiais.001 vertice inicial:  1263057
Materiais vertice inicial:  1350753
Processando modelo arvore.obj. Vertice final: 1408281


In [126]:
modelo = load_model_from_file('pato/pato.obj')

print('Processando modelo pato.obj. Vertice inicial:',len(vertices_list))
pato_text_id = []
pato_vertices_text = []
ini_pato = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo pato.obj. Vertice final:',len(vertices_list))
fim_pato = len(vertices_list)

pato_vertices_text.append(ini_pato)
pato_vertices_text.append(fim_pato)

load_texture_from_file(prox_textura,'pato/pato.jpg')
increase_text(pato_text_id)

Processando modelo pato.obj. Vertice inicial: 1408281
Processando modelo pato.obj. Vertice final: 1460793


In [127]:
modelo = load_model_from_file('mesa/mesa.obj')

print('Processando modelo mesa.obj. Vertice inicial:',len(vertices_list))
mesa_text_id = []
mesa_vertices_text = []
ini_mesa = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo mesa.obj. Vertice final:',len(vertices_list))
fim_mesa = len(vertices_list)

mesa_vertices_text.append(ini_mesa)
mesa_vertices_text.append(fim_mesa)

load_texture_from_file(prox_textura,'mesa/mesa.png')
increase_text(mesa_text_id)

Processando modelo mesa.obj. Vertice inicial: 1460793
Processando modelo mesa.obj. Vertice final: 1469547


In [128]:
modelo = load_model_from_file('ceu/ceu.obj')

print('Processando modelo ceu.obj. Vertice inicial:',len(vertices_list))
ceu_text_id = []
ceu_vertices_text = []
ini_ceu = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo ceu.obj. Vertice final:',len(vertices_list))
fim_ceu = len(vertices_list)

ceu_vertices_text.append(ini_ceu)
ceu_vertices_text.append(fim_ceu)

load_texture_from_file(prox_textura,'ceu/ceu.png')
increase_text(ceu_text_id)

Processando modelo ceu.obj. Vertice inicial: 1469547
Processando modelo ceu.obj. Vertice final: 1472427


In [129]:
modelo = load_model_from_file('carro/carro.obj')

print('Processando modelo carro.obj. Vertice inicial:',len(vertices_list))
carro_text_id = []
carro_vertices_text = []
ini_carro = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo carro.obj. Vertice final:',len(vertices_list))
fim_carro = len(vertices_list)

carro_vertices_text.append(ini_carro)
carro_vertices_text.append(fim_carro)

load_texture_from_file(prox_textura,'carro/carro.jpg')
increase_text(carro_text_id)

Processando modelo carro.obj. Vertice inicial: 1472427
Processando modelo carro.obj. Vertice final: 1486629


In [130]:
modelo = load_model_from_file('cachorro/cachorro.obj')

print('Processando modelo cachorro.obj. Vertice inicial:',len(vertices_list))
cachorro_text_id = []
cachorro_vertices_text = []
ini_cachorro = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cachorro.obj. Vertice final:',len(vertices_list))
fim_cachorro = len(vertices_list)

cachorro_vertices_text.append(ini_cachorro)
cachorro_vertices_text.append(fim_cachorro)

load_texture_from_file(prox_textura,'cachorro/cachorro.jpg')
increase_text(cachorro_text_id)

Processando modelo cachorro.obj. Vertice inicial: 1486629
Processando modelo cachorro.obj. Vertice final: 1702533


In [131]:
modelo = load_model_from_file('homem/homem.obj')

print('Processando modelo homem.obj. Vertice inicial:',len(vertices_list))
homem_text_id = []
homem_vertices_text = []
ini_homem = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo homem.obj. Vertice final:',len(vertices_list))
fim_homem = len(vertices_list)

homem_vertices_text.append(ini_homem)
homem_vertices_text.append(fim_homem)

load_texture_from_file(prox_textura,'homem/homem.jpg')
increase_text(homem_text_id)

Processando modelo homem.obj. Vertice inicial: 1702533
Processando modelo homem.obj. Vertice final: 1764159


In [132]:
modelo = load_model_from_file('cama/cama.obj')

print('Processando modelo cama.obj. Vertice inicial:',len(vertices_list))
cama_text_id = []
cama_vertices_text = []
ini_cama = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cama.obj. Vertice final:',len(vertices_list))
fim_cama = len(vertices_list)

cama_vertices_text.append(ini_cama)
cama_vertices_text.append(fim_cama)

load_texture_from_file(prox_textura,'cama/cama.tga')
increase_text(cama_text_id)

Processando modelo cama.obj. Vertice inicial: 1764159
Processando modelo cama.obj. Vertice final: 2396676


In [133]:
modelo = load_model_from_file('parede/wall.obj')

print('Processando modelo parede.obj. Vertice inicial:',len(vertices_list))
parede_text_id = []
parede_vertices_text = []
ini_parede = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo parede.obj. Vertice final:',len(vertices_list))
fim_parede = len(vertices_list)

parede_vertices_text.append(ini_parede)
parede_vertices_text.append(fim_parede)

load_texture_from_file(prox_textura,'parede/parede.jpg')
increase_text(parede_text_id)

Processando modelo parede.obj. Vertice inicial: 2396676
Processando modelo parede.obj. Vertice final: 2396748


In [134]:
modelo = load_model_from_file('arvore/arvore-2.obj')

print('Processando modelo arvore-2.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
arvore2_vertices_text = []
arvore2_text_id = []
ini_arvore2 = len(vertices_list)
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], 'vertice inicial: ', len(vertices_list))
        faces_visited.append(face[2])
        arvore2_vertices_text.append(len(vertices_list))
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo arvore-2.obj. Vertice final:',len(vertices_list))
fim_arvore2 = len(vertices_list)
    
arvore2_vertices_text.append(fim_arvore2)

load_texture_from_file(prox_textura,'arvore/tronco-2.png')
increase_text(arvore2_text_id)
load_texture_from_file(prox_textura,'arvore/folha-2.png')
increase_text(arvore2_text_id)

Processando modelo arvore-2.obj. Vertice inicial: 2396748
Materiais.002 vertice inicial:  2396748
Materiais.003 vertice inicial:  2506188
Processando modelo arvore-2.obj. Vertice final: 2584644


In [135]:
modelo = load_model_from_file('caveira/caveira.obj')

print('Processando modelo caveira.obj. Vertice inicial:',len(vertices_list))
caveira_text_id = []
caveira_vertices_text = []
ini_caveira = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo caveira.obj. Vertice final:',len(vertices_list))
fim_caveira = len(vertices_list)

caveira_vertices_text.append(ini_caveira)
caveira_vertices_text.append(fim_caveira)

load_texture_from_file(prox_textura,'caveira/caveira.jpg')
increase_text(caveira_text_id)

Processando modelo caveira.obj. Vertice inicial: 2584644
Processando modelo caveira.obj. Vertice final: 2824692


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [136]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)

###  Enviando coordenadas de vértices para a GPU

In [137]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

###  Enviando coordenadas de textura para a GPU

In [138]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [139]:
def desenha_arvore1(_tx, _ty, _tz):
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = _tx; ty = _ty; tz = _tz;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    for i in range(0, len(arvore1_text_id)):
        glBindTexture(GL_TEXTURE_2D, arvore1_text_id[i])
        glDrawArrays(GL_TRIANGLES, arvore1_vertices_text[i], arvore1_vertices_text[i+1]-arvore1_vertices_text[i])
    

In [140]:
def desenha_terreno():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = 0.0; ty = -1.01; tz = 0.0;
    
    # escala
    sx = 40.0; sy = 40.0; sz = 40.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(pedra_text_id)):
        glBindTexture(GL_TEXTURE_2D, pedra_text_id[i])
        glDrawArrays(GL_TRIANGLES, pedra_vertices_text[i], pedra_vertices_text[i+1]-pedra_vertices_text[i])

    for i in range(0, len(grama_text_id)):
        glBindTexture(GL_TEXTURE_2D, grama_text_id[i])
        glDrawArrays(GL_TRIANGLES, grama_vertices_text[i], grama_vertices_text[i+1]-grama_vertices_text[i])

In [141]:
def desenha_sofa():
    
    # rotacao
    angle = 270.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 18.5; ty = -0.9; tz = -5.0;
    
    # escala
    sx = 0.6; sy = 0.6; sz = 0.6;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    for i in range(0, len(sofa_text_id)):
        glBindTexture(GL_TEXTURE_2D, sofa_text_id[i])
        glDrawArrays(GL_TRIANGLES, sofa_vertices_text[i], sofa_vertices_text[i+1]-sofa_vertices_text[i])

In [142]:
def desenha_casa():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = -8.0; ty = -1.0; tz = 5.0;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    for i in range(0, len(casa_text_id)):
        glBindTexture(GL_TEXTURE_2D, casa_text_id[i])
        glDrawArrays(GL_TRIANGLES, casa_vertices_text[i], casa_vertices_text[i+1]-casa_vertices_text[i])

In [143]:
def desenha_tv():
    
    # rotacao
    angle = -270.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = -3.5; ty = 1.1; tz = -4.5;
    
    # escala
    sx = 0.6; sy = 0.6; sz = 0.6;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(tv_text_id)):
        glBindTexture(GL_TEXTURE_2D, tv_text_id[i])
        glDrawArrays(GL_TRIANGLES, tv_vertices_text[i], tv_vertices_text[i+1]-tv_vertices_text[i])

In [144]:
def desenha_pato():
    
    global inc_x, dec_z, tx_tz, angle_duck
    
    # rotacao
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = inc_x; ty = -1.0; tz = dec_z;
    
    if inc_x > 25.0:
        inc_x = 25.0
        tx_tz = [True, False]
        angle_duck += 90.0
    elif inc_x < -20.0:
        inc_x = -20.0
        tx_tz = [True, True]
        angle_duck += 90.0
    elif inc_x < 25.0 and tx_tz == [False, False]:
        inc_x += 0.1
    elif inc_x > -20.0 and tx_tz == [False, True]:
        inc_x -= 0.1
    
    if dec_z < -35.0:
        dec_z = -35.0
        tx_tz = [False, True]
        angle_duck += 90.0
    elif dec_z > 9.0:
        dec_z = 9.0
        tx_tz = [False, False]
        angle_duck += 90.0
    elif dec_z > -35.0 and tx_tz == [True, False]:
        dec_z -= 0.1
    elif dec_z < 9.0 and tx_tz == [True, True]:
        dec_z += 0.1
    
    # escala
    sx = 0.3; sy = 0.3; sz = 0.3;
    
    mat_model = model(angle_duck, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(pato_text_id)):
        glBindTexture(GL_TEXTURE_2D, pato_text_id[i])
        glDrawArrays(GL_TRIANGLES, pato_vertices_text[i], pato_vertices_text[i+1]-pato_vertices_text[i])

In [145]:
def desenha_mesa():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = -4.0; ty = -1.0; tz = -4.5;
    
    # escala
    sx = 6.0; sy = 6.0; sz = 6.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(mesa_text_id)):
        glBindTexture(GL_TEXTURE_2D, mesa_text_id[i])
        glDrawArrays(GL_TRIANGLES, mesa_vertices_text[i], mesa_vertices_text[i+1]-mesa_vertices_text[i])

In [146]:
def desenha_ceu():
    
    # rotacao
    angle = 90.0;
    rx = 90.0; ry = 1.0; rz = 90.0;
    
    # translacao
    tx = 40.0; ty = -60.0; tz = -100.5;
    
    # escala
    sx = 20.0; sy = 20.0; sz = 20.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(ceu_text_id)):
        glBindTexture(GL_TEXTURE_2D, ceu_text_id[i])
        glDrawArrays(GL_TRIANGLES, ceu_vertices_text[i], ceu_vertices_text[i+1]-ceu_vertices_text[i])

In [147]:
def desenha_carro():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = -28; ty = -1.0; tz = -5.0;
    
    # escala
    sx = 4.0; sy = 4.0; sz = 4.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    for i in range(0, len(carro_text_id)):
        glBindTexture(GL_TEXTURE_2D, carro_text_id[i])
        glDrawArrays(GL_TRIANGLES, carro_vertices_text[i], carro_vertices_text[i+1]-carro_vertices_text[i])

In [148]:
def desenha_cachorro():
    
    # rotacao
    angle = 0.0;
    rx = -5.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 30; ty = -1.0; tz = 9.0;
    
    # escala
    sx = 0.15; sy = 0.15; sz = 0.15;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(cachorro_text_id)):
        glBindTexture(GL_TEXTURE_2D, cachorro_text_id[i])
        glDrawArrays(GL_TRIANGLES, cachorro_vertices_text[i], cachorro_vertices_text[i+1]-cachorro_vertices_text[i])

In [149]:
def desenha_homem():

    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 1.0; rz = 0.0;

    # translacao
    tx = -1.0; ty = -1.0; tz = -15.0;

    # escala
    sx = 0.4; sy = 0.4; sz = 0.4;

    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    for i in range(0, len(homem_text_id)):
        glBindTexture(GL_TEXTURE_2D, homem_text_id[i])
        glDrawArrays(GL_TRIANGLES, homem_vertices_text[i], homem_vertices_text[i+1]-homem_vertices_text[i])

In [150]:
def desenha_cama():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 14.5; ty = -1.7; tz = -18.0;
    
    # escala
    sx = 0.05; sy = 0.05; sz = 0.05;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(cama_text_id)):
        glBindTexture(GL_TEXTURE_2D, cama_text_id[i])
        glDrawArrays(GL_TRIANGLES, cama_vertices_text[i], cama_vertices_text[i+1]-cama_vertices_text[i])

In [151]:
def desenha_parede(tx, ty, tz, angle):
    
    # rotacao
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(parede_text_id)):
        glBindTexture(GL_TEXTURE_2D, parede_text_id[i])
        glDrawArrays(GL_TRIANGLES, parede_vertices_text[i], parede_vertices_text[i+1]-parede_vertices_text[i])

In [152]:
def desenha_arvore2(_tx, _ty, _tz):
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = _tx; ty = _ty; tz = _tz;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    for i in range(0, len(arvore2_text_id)):
        glBindTexture(GL_TEXTURE_2D, arvore2_text_id[i])
        glDrawArrays(GL_TRIANGLES, arvore2_vertices_text[i], arvore2_vertices_text[i+1]-arvore2_vertices_text[i])

In [153]:
def desenha_caveira(rotacao_inc):
    
    # rotacao
    angle = rotacao_inc;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 0.0; ty = 2.0; tz = -75.0;
    
    # escala
    sx = 0.7; sy = 0.7; sz = 0.7;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    for i in range(0, len(caveira_text_id)):
        glBindTexture(GL_TEXTURE_2D, caveira_text_id[i])
        glDrawArrays(GL_TRIANGLES, caveira_vertices_text[i], caveira_vertices_text[i+1]-caveira_vertices_text[i])

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [154]:
cameraPos   = glm.vec3(10.0,  10.0,  10.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);
near = 0.1
far = 200.0
fovy = 45.0

def higher_than_ground(pressedW, pressedS, pressedA, pressedD):
    auxW = cameraPos + cameraSpeed * cameraFront
    auxS = cameraPos - cameraSpeed * cameraFront
    
    if(pressedW and auxW[1] > -1):
        return True
    elif(pressedS and auxS[1] > -1):
        return True
    else:
        return False

def inside_wall(pressedW, pressedS, pressedA, pressedD):
    auxW = cameraPos + cameraSpeed * cameraFront
    auxS = cameraPos - cameraSpeed * cameraFront
    auxA = cameraPos - glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    auxD = cameraPos + glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if(pressedW and auxW[2] > -48.25 and auxW[2] < 38.25 and auxW[0] > -48 and auxW[0] < 48 and auxW[1] < 22.6117):
        return True
    elif(pressedS and auxS[2] > -48.25 and auxS[2] < 38.25 and auxS[0] > -48 and auxS[0] < 48 and auxS[1] < 22.6117):
        return True
    elif(pressedA and auxA[2] > -48.25 and auxA[2] < 38.25 and auxA[0] > -48 and auxA[0] < 48 and auxA[1] < 22.6117):
        return True
    elif(pressedD and auxD[2] > -48.25 and auxD[2] < 38.25 and auxD[0] > -48 and auxD[0] < 48 and auxD[1] < 22.6117):
        return True
    
    return False

polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, cameraSpeed, polygonal_mode, far, near, fovy
    
    cameraSpeed = 1.5

    if key == 70 and (action==1 or action==2): #F FAR
        if far - 1.0 > 0.0:
            far -= 1.0
    if key == 71 and (action==1 or action==2): #G
        if far + 1.0 < 1000.0:
            far += 1.0
    if key == 78 and (action==1 or action==2): #N NEAR
        if near - 1.0 > 0.0:
            near -= 1.0
    if key == 77 and (action==1 or action==2): #M
        if near + 1.0 < 1000.0:
            near += 1.0
    if key == 75 and (action==1 or action==2): #K FOV
        if fovy - 1.0 > 0.0:
            fovy -= 1.0
    if key == 76 and (action==1 or action==2): #L
        if fovy + 1.0 < 100.0:
            fovy += 1.0
    if key == 87 and (action==1 or action==2): # tecla W
        if higher_than_ground(True, False, False, False) and inside_wall(True, False, False, False):
            cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        if higher_than_ground(False, True, False, False) and inside_wall(False, True, False, False):
            cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        if inside_wall(False, False, True, False):
            cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        if inside_wall(False, False, False, True):
            cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [155]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura, near, far, fovy
    
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(fovy), largura/altura, near, far)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [156]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [157]:
glEnable(GL_DEPTH_TEST)
global inc_x, dec_z, tx_tz, angle_duck
angle_duck = -15.0
tx_tz = [False, False]
inc_x = -20.0   
dec_z = 9.0
rotacao_inc = 0.0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    desenha_arvore1(40.0, -1.0, 30.0)  
    desenha_arvore2(25.0, -1.0, 30.0)
    desenha_arvore1(10.0, -1.0, 30.0)
    desenha_arvore2(-5.0, -1.0, 30.0)   
    desenha_arvore1(-20.0, -1.0, 30.0)
    desenha_arvore2(-35.0, -1.0, 30.0)
    desenha_terreno()
    desenha_casa()
    desenha_sofa()
    desenha_tv()
    desenha_pato()
    desenha_mesa()
    desenha_ceu()
    desenha_carro()
    desenha_cachorro()
    desenha_homem()
    desenha_cama()
    desenha_caveira(rotacao_inc)
    desenha_parede(50.0, -1.7, 20.25, 90)
    desenha_parede(50.0, -1.7, -20.25, 90)
    desenha_parede(50.0, -1.7, 60.75, 90)
    desenha_parede(20.25, -1.7, -50.25, 0)
    desenha_parede(-20.25, -1.7, -50.25, 0)
    desenha_parede(-60.75, -1.7, -50.25, 0)
    desenha_parede(-50.0, -1.7, 20.25, 90)
    desenha_parede(-50.0, -1.7, -20.25, 90)
    desenha_parede(-50.0, -1.7, 60.75, 90)
    desenha_parede(20.25, -1.7, 40.25, 0)
    desenha_parede(-20.25, -1.7, 40.25, 0)
    desenha_parede(-60.75, -1.7, 40.25, 0)
    #x = 48 -48
    #z = -48.25 38.25
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    rotacao_inc += 0.1
    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    

    
    glfw.swap_buffers(window)

glfw.terminate()